# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import keras
import pandas as pd

In [2]:
data = pd.read_csv("tic-tac-toe.csv")
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
categorical = data.select_dtypes(exclude='number')
for column in list(categorical):
    data[column] = LabelEncoder().fit_transform(data[column])

In [4]:
X = data.drop(columns=['class'])
y = data['class']

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [6]:
model = Sequential()

In [7]:
# Input layer: Matches the size of your input data (flattened Tic-Tac-Toe board)
model.add(Dense(40, activation='relu')) 

# Hidden layer(s): Experiment with the number of neurons and layers
model.add(Dense(40, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(40, activation='relu'))

# Output layer: Number of neurons = number of classes (possible outcomes in Tic-Tac-Toe)
model.add(Dense(3, activation='softmax')) 

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test)) 

Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - accuracy: 0.6320 - loss: 0.9626 - val_accuracy: 0.6562 - val_loss: 0.7081
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6732 - loss: 0.6609 - val_accuracy: 0.6615 - val_loss: 0.6276
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6413 - loss: 0.6288 - val_accuracy: 0.6615 - val_loss: 0.6031
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6599 - loss: 0.6099 - val_accuracy: 0.6615 - val_loss: 0.6008
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6717 - loss: 0.5977 - val_accuracy: 0.7448 - val_loss: 0.5701
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7443 - loss: 0.5536 - val_accuracy: 0.7552 - val_loss: 0.5474
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7405 - loss: 0.5612 - val_accuracy: 0.7708 - val_loss: 0.5291
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7656 - loss: 0.5182 - val_accuracy: 0

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Test accuracy: 0.90625
Test loss: 0.39043334126472473


In [11]:
model.save('tic-tac-toe.model.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [12]:
model = load_model('tic-tac-toe.model.keras') 

In [13]:
prediction = model.predict([X_test])

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


In [14]:
model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


array([[3.74276283e-06, 9.99996185e-01, 1.00475887e-07],
       [9.30299878e-01, 6.96996972e-02, 4.77364779e-07],
       [1.00000000e+00, 3.58286995e-13, 1.27252801e-14],
       [1.00000000e+00, 5.98939609e-09, 3.08856517e-11],
       [3.12566328e-07, 9.99999642e-01, 2.76812635e-08],
       [9.88651276e-01, 1.13482466e-02, 5.16907960e-07],
       [9.22536012e-03, 9.90772247e-01, 2.44111789e-06],
       [9.99999642e-01, 3.23838151e-07, 4.42773707e-10],
       [1.39539884e-02, 9.86043930e-01, 2.13738508e-06],
       [9.87796605e-01, 1.22024873e-02, 9.24563039e-07],
       [9.99996901e-01, 3.07422579e-06, 1.63591647e-08],
       [4.19396628e-07, 9.99999523e-01, 1.09986978e-07],
       [9.99998093e-01, 1.94400445e-06, 3.80076548e-09],
       [1.07764319e-01, 8.92226458e-01, 9.25636596e-06],
       [2.65365649e-08, 1.00000000e+00, 1.51680790e-09],
       [1.86422335e-06, 9.99998093e-01, 6.18992786e-08],
       [3.24492212e-05, 9.99966025e-01, 1.50138703e-06],
       [2.47921753e-06, 9.99996

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [143]:
for i in [0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]:
  print(i)

  model = Sequential()
  
  # Input layer: Matches the size of your input data (flattened Tic-Tac-Toe board)
  model.add(Dense(40, activation='relu'))
  
  # Hidden layer(s): Experiment with the number of neurons and layers
  model.add(Dense(40, activation='relu'))
  model.add(Dense(40, activation='relu'))
  model.add(Dense(40, activation='relu'))
  
  # Output layer: Number of neurons = number of classes (possible outcomes in Tic-Tac-Toe)
  model.add(Dense(3, activation='softmax'))
  
  opt = keras.optimizers.Adam(learning_rate=i)
  
  model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
  model.fit(X_train, y_train, epochs=95, batch_size=32, validation_data=(X_test, y_test), verbose=False) 
  
  test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
  print('Test accuracy:', test_acc)
  print('Test loss:', test_loss)

0.001
Test accuracy: 0.890625
Test loss: 0.31355908513069153


Test accuracy: 0.8958333134651184
Test loss: 0.34641408920288086

**Which approach(es) did you find helpful to improve your model performance?**

In [24]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Create the optimizer instance

optimizer = Adam(learning_rate=0.01)

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=110, batch_size=17, validation_data=(X_test, y_test), verbose=False)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(loss, accuracy)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9808 - loss: 0.0648 
0.06850503385066986 0.9739583134651184


epoch 120 | batch 24: 0.21556060016155243 0.9635416865348816

epoch 120 | batch 18: 0.14408843219280243 0.9791666865348816

epoch 120 | batch 17: 0.10755687206983566 0.9739583134651184

epoch 110 | batch 17: 0.06850503385066986 0.9739583134651184
